In [1]:
from modules import *
from functions import *

In [10]:
# Parameters

N = int(500)                                                        # Number of neurons
a = 10                                                                # Parameters for propensity function
theta_stim = 90                                                     # Angle to stimulate at 
n_test_angles = 100                                                 # Number of angles to use to test for preferred orientation
vars = np.random.lognormal(2, 0.6, N)                               # Width of each neuron's tuning curve
learning_rate = 0.01                                                # Learning rate

n_days = 28                                                         # Number of days to run for
n_norm_per_day = 1                                                  # How many times to normalise the weights per day
n_steps_per_norm = 30                                               # How many orientation stimuli per day                    *** Increase this (and decrease learning rate respectively) to simulate more visual experience per day. Note: will take longer to run ***
n_steps = n_steps_per_norm * n_norm_per_day * n_days                # Number of steps to run for
init_steps = 300                                                    # Number of trials to run to settle to a baseline

hebb_scaling = 0.3                                                  # Scaling of Hebbian component
rand_scaling = 1                                                    # Scaling of random component 

# Initialisation

W_init = initialise_W(N, vars)                                                                                                      # Initialise weights 
W_baseline = prerun(W_init, theta_stim, a, hebb_scaling, rand_scaling, learning_rate, n_steps_per_norm, init_steps)                 # Settle to a baseline

POs = []; ratios = []
W = np.zeros((N, N, n_steps+1), dtype=np.float32)                                                                                   # Initialise weight matrix
W[:, :, 0] = W_baseline
W_per_day = np.zeros((N, N, n_days))

# Run trials

for t in tqdm(range(n_steps)):
    W_old = W[:, :, t]
    H = single_hebbian_component(N, W_old, theta_stim, type='stripe_rearing')                    # Hebbian component - outer product of pre- and post-synaptic activity      *** Change type from 'baseline' to 'stripe_rearing' to switch between conditions ***
    eta = np.random.randn(N, N)                                                                  # Random component - sample from normal distribution
    prop_function = propensity(W_old, a)                                                         # Propensity function - tanh(w)
    hebb =  hebb_scaling * H                                                                     # Scaled Hebbian component 
    rand =  rand_scaling * eta                                                                   # Scaled random component
    W_new = W_old + learning_rate * prop_function * (hebb + rand)                                # Update weights

    if t % n_steps_per_norm == 0:                                                                # Perform normalisation after every N_theta steps 
        normalisation(W_new)
        if t % (n_steps_per_norm * n_norm_per_day) == 0:                                         # Save weights and preferred orientations every day 
            W_per_day[:, :, t // (n_steps_per_norm * n_norm_per_day)] = W_new                    
            PO = get_preferred_orientations(N, W_new, n_angles=n_test_angles); POs.append(PO)    
            ratios.append(np.mean(abs(hebb)) / (np.mean(abs(rand)) + 10e-10))

    W[:, :, t+1] = W_new

preferences = np.array(POs).T 
initial_preferences = np.linspace(0, 180, N) 
final_preferences = preferences[:, -1]
absolute_change = np.array([circular_distance(preferences[:, day], initial_preferences) for day in range(n_days)])
median_change_per_day = np.median(absolute_change, axis=1)   
std_change_per_day = np.std(absolute_change, axis=1)  

100%|██████████| 840/840 [00:21<00:00, 39.58it/s]


## Fig. 3c

In [11]:
def fig_3c(preferences, final_day=20):
    """ run with data from baseline conditions """
    fig, ax = plt.subplots(figsize=(2.5, 2.5), dpi=180)
    ax.scatter(preferences[:, 0], preferences[:, final_day], s=20, alpha=0.4, fc='k', ec='None')
    ax.set_ylabel(r'PO day n + 20  $[\degree]$')
    ax.set_xlabel(r'PO day n  $[\degree]$')
    ax.set_xticks([0, 90, 180]); ax.set_yticks([0, 90, 180])
    ax.set_xticklabels(['-90', '0', '90']); ax.set_yticklabels(['-90', '0', '90'])
    ax.set_aspect('equal')
    fig.show()

## Fig. 3g

In [13]:
initial_distances = np.abs(initial_preferences - theta_stim)
final_distances = np.abs(final_preferences - theta_stim)
change = initial_distances - final_distances
total_drift = circular_distance(initial_preferences, final_preferences)

def fig_3g(initial_distances, total_drift):
    """ run with data from deprivation conditions """
    fig, ax = plt.subplots(figsize=(2.5, 2.5), dpi=180)
    ax.scatter(initial_distances, total_drift, s=20, color='k', ec='None', clip_on=False, alpha=0.4)
    ax.set_yticks(np.arange(0, 91, 30)); ax.set_xticks(np.arange(0, 91, 30))
    ax.set_xlim(0, 90); ax.set_ylim(0, 90)
    ax.set_xlabel(r'initial |relative PO|  $[\degree]$')
    ax.set_ylabel(r'drift magnitude  $[\degree]$')
    fig.show()

## Supp. Fig. 3g

In [31]:
def supp_fig_3g(initial_distances, change):
    """ run with data from deprivation conditions """

    fig = plt.figure(figsize=(3.2, 2.5), dpi=180)

    gs = fig.add_gridspec(1, 2,  width_ratios=(4, 1),  left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.2, hspace=0.05)
    ax = fig.add_subplot(gs[0])
    ax_hist = fig.add_subplot(gs[1], sharey=ax)

    ax.fill_between(np.arange(0, 91, 1), np.arange(0, 91, 1), 90, color='lightgray', alpha=0.5) 
    ax.fill_between(np.arange(0, 90, 1), np.arange(-90, 0, 1), -90, color='lightgray', alpha=0.5)
    ax.axhline(0, ls='--', color='k')
    ax_hist.axhline(0, ls='--', color='k')

    ax.scatter(initial_distances, change, s=20, color='k', ec='None', clip_on=False, alpha=0.4)
    ax_hist.hist(change, bins=17, orientation="horizontal", histtype="stepfilled", alpha=0.4, ec='k', fc='k')

    xlim = ax_hist.get_xlim()[1]
    ax_hist.set_xticks([0, xlim])
    ax_hist.set_xticklabels([0, np.round(xlim / N, 1)])
    ax_hist.set_xlabel('Fraction')
    ax_hist.tick_params(axis="y", labelleft=False)

    ax.set_yticks(np.arange(-90, 91, 30)); ax.set_xticks(np.arange(0, 91, 30))
    ax.set_xlim(0, 90); ax.set_ylim(-90, 90); ax.set_yticks(np.arange(-90, 91, 90))
    ax.set_xlabel(r'initial |relative PO|  $[\degree]$')
    ax.set_ylabel(r'convergence \; $[\degree]$')
    fig.show()

## Supp. Fig. 3f

In [32]:
drift_rates = np.mean(np.array([circular_distance(preferences[:, x+1], preferences[:, x]) for x in range(n_days-1)]), axis=0)

def supp_fig_3f(initial_distances, drift_rates):
    """ run with data from deprivation conditions """

    fig, ax = plt.subplots(figsize=(2.3, 2.3), dpi=180)
    ax.scatter(initial_distances, drift_rates, s=20, color='k', ec='None', clip_on=False, alpha=0.4)
    ax.set_yticks(np.arange(0, 91, 10))
    ax.set_xticks(np.arange(0, 91, 30))
    ax.set_xlim(0, 90); ax.set_ylim(0, 30)
    ax.set_xlabel(r'initial |relative PO|  $[\degree]$')
    ax.set_ylabel(r'drift rate  $[\degree / day]$')
    fig.show()

## Supp. Fig. 3d

In [35]:
def supp_fig_3d(vars, drift_rates):
    """ run with data from baseline condition """

    fig, ax = plt.subplots(figsize=(2.3, 2.3), dpi=180)
    ax.scatter(vars, drift_rates, s=15, color='k', ec='None', clip_on=False, alpha=0.5)
    ax.set_xscale('log'); ax.set_yscale('log')
    ax.set_xlabel(r'tuning curve width  $ \; [\degree]$')
    ax.set_ylabel(r'drift rate $ \; [\degree]$')
    fig.show()

## Supp. Fig. 3e

In [36]:
def supp_fig_3e(absolute_change):
    """ run with data from baseline condition """

    fig, ax = plt.subplots(figsize=(2.3, 2.1), dpi=180)

    cmap = plt.get_cmap('viridis')
    colors = [cmap(i) for i in np.linspace(0, 1, n_days)]

    for i in range(0, n_days-1, 1):
        count, bins_count = np.histogram(absolute_change[i], bins=100)
        pdf = count / sum(count); cdf = np.cumsum(pdf)
        ax.plot(bins_count[1:], cdf, c=colors[i], label='Day {}'.format(i+1), clip_on=False)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=n_days))
    cbar = plt.colorbar(sm, ax=ax, ticks=np.arange(0, n_days+5, 5))
    cbar.ax.set_yticklabels(np.arange(0, n_days+5, 5))
    cbar.set_label('Days', rotation=270, labelpad=20, fontsize=12)
    ax.set_xlim([0, 80]); ax.set_xticks(np.arange(0, 81, 20))
    ax.set_ylim([0, 1]); ax.set_yticks(np.arange(0, 1.1, 0.5))

    ax.set_xlabel(r'drift magnitude  $[\degree]$')
    ax.set_ylabel('cumulative prob.')
    fig.show()

## Fig. 3i

In [47]:
PO = preferences
PO[:, 0] = np.linspace(0, 180, N)
dPO = []                                          # Change in preference at each time step 
rPO = []                                          # Relative preference at each time step
dPO_ShM = []                                      # Shuffled magnitude
dPO_ShD = []                                      # Shuffled direction

for i in range(n_days): 
    dPO.append(directional_circular_distance(PO[:,i], PO[:,0]))
    rPO.append(PO[:,i] - theta_stim)
    dPO_ShM.append(np.random.permutation(np.abs(dPO[i])) * (np.sign(dPO[i])+(dPO[i]==0)))   # Permute the value, keep the sign 
    dPO_ShD.append(np.random.permutation(np.sign(dPO[i])+(dPO[i]==0)) * np.abs(dPO[i]))     # Permute the sign, keep the value
    sorted_dPO = np.sort(np.abs(dPO[i]))
    order_start_rPO = np.argsort(np.abs(rPO[0]))

dPO, rPO, dPO_ShM, dPO_ShD = convert_to_array(dPO, rPO, dPO_ShM, dPO_ShD)

# Final preferences after shuffling 
PO_ShM = (PO[:,0]+dPO_ShM)  
PO_ShD = (PO[:,0]+dPO_ShD)

PO_ShM, PO_ShD = convert_to_array(PO_ShM, PO_ShD)
rPO_ShM, rPO_ShD = [directional_circular_distance(x, theta_stim) for x in (PO_ShM, PO_ShD)]   # Relative distances after shuffling
drPO, drPO_ShM, drPO_ShD = [np.abs(rPO[0]) - np.abs(x) for x in (rPO, rPO_ShM, rPO_ShD)]      # Change in relative distance after shuffling


data_means = [2.9706, 0.2923,	2.3746]              # Values of experimental data (Fig. 2g and 2i)
data_lower_bound = [0.5411,	1.3523, 0.7938]
data_upper_bound = [1.1688, 0.951, 	0.914]


def fig_3i(drPO, drPO_ShM, drPO_ShD, data_means, data_lower_bound, data_upper_bound):
    """ run with data from deprivation conditions """

    fig, ax = plt.subplots(1, 1, figsize=(1.4, 2.3), dpi=180)
    ax.scatter([0, 1, 2], data_means, c='darkgrey', alpha=1, zorder=3)
    ax.errorbar([0, 1, 2], data_means, yerr=[data_lower_bound, data_upper_bound], fmt='o', c='darkgrey', alpha=1, capsize=3)
    ax.scatter([0.2, 1.2, 2.2], [np.median(drPO[-1]), np.median(drPO_ShD[-1]), np.median(drPO_ShM[-1])],  c='green', zorder=3)
    ax.set_xticks([0, 1, 2], ['model', 'shuf dir.', 'shuf mag.'], rotation=30);
    ax.set_ylabel(r'convergence  $[\degree]$')
    ax.axhline(0, c='k', ls='--', alpha=0.3, lw=2)
    ax.set_xlim([-0.5, 2.5]); ax.set_ylim([-2, 5])
    fig.show()